# Setup
1. Install pymongo in your python install 
2. Sign up for an account at mlab.com
2. Choose the free "development" version
3. Choose a db name for this test version
4. Once the db is created, click on the "users" tab and create at least 1 user for this db
5. With the db setup and a user created, you can now use the code below to get started with a MongoDB

For info on mongodb, see https://docs.mongodb.com/v3.2/tutorial/

In [2]:
import pymongo

connection = pymongo.MongoClient("ds031925.mlab.com",31925)
db = connection["f16_class_demo"]
user = "admin"
pwd = "thisisatest"
db.authenticate(user, pwd) # if this works, you should get a True result, False otherwise

True

In [3]:
#let's create a new user
db.users.insert_one({
  "class": "test"
})  #this creates a collection called, "users" and inserts a document into it

In [224]:
#let's insert a record into the db.  Records are stored as python dictionaries
import datetime
blogPost = {"author":"Larry",
            "title":"Flippin Flappin Plumbing",
            "tags":["mongodb","demo","examples","760"],
            "date":datetime.datetime.utcnow()
    }

posts = db.log # note that if this collection doesn't yet exist, mongo will create it
posts_id = posts.insert_one(blogPost)


In [225]:
import random
import time

from datetime import timedelta
from random import randint
import datetime

#use this to generate a random date between two ranges
## formula came from http://stackoverflow.com/questions/553303/generate-a-random-date-between-two-other-dates
def random_date(start, end):
    return start + timedelta(seconds=randint(0, int((end - start).total_seconds())))

startDate = datetime.datetime.strptime('2016-10-01','%Y-%m-%d').date()
endDate = datetime.datetime.strptime('2016-11-04','%Y-%m-%d').date()

print(random_date(startDate,endDate))

2016-10-27


In [228]:
#just for fun, let's create a lot of posts, so that we can search through them
## we'll first create a several lists so that we can create unique posts
authors = ["Mike","Carol","Alice","Greg","Marcia","Peter","Jan","Bobby","Cindy"]
tags = ["demo","rant","insight","stupid","important","political","personal","760R","BYU","fun","work","random"]
titleArticle = ["The","A"]
titleNouns = ["discussion","sea","nation","election","effect","battle","discovery","comparison","industry","governor","match","kittens","animals","year"]
titleAdverbs = ["about","where","very","how","just","now","even","very","only","never"]
titlePrepositions = ["of","around","in","on","about","through","inside","with","at","within","without","under","by"]
titleNounsPlural = ["candidates","homes","suits","trains","books","ice","snakes","deer","scenes","weather","fruit","horses","airplanes","streets","yams","cables","schools","yokes"]
titleVerbs = ["relax","bomb","mourn","wipe","slow","borrow","flap","pour","welcome","remember","irritate","use","unfasten","agree","share","choke","argue","tire","squeal","unfasten","box","launch","scratch","use","stretch"]


import random
import time

from datetime import timedelta
from random import randint
import datetime

#use this to generate a random date between two ranges
## formula came from http://stackoverflow.com/questions/553303/generate-a-random-date-between-two-other-dates
def random_date(start, end):
    return start + timedelta(seconds=randint(0, int((end - start).total_seconds())))

startDate = datetime.datetime.strptime('2016-10-01','%Y-%m-%d').date()
endDate = datetime.datetime.strptime('2016-11-04','%Y-%m-%d').date()

posts = db.log

for i in range(1,100):
    numTags = random.randint(1,4)
    thisPostTags = []
    for j in range(1,4):
        thisWord = tags[random.randint(0,len(tags)-1)]
        if thisWord not in thisPostTags:
            thisPostTags.append(thisWord)
    thisDate = random_date(startDate,endDate)
    thisPost = {
        "author":authors[random.randint(0,len(authors)-1)],
        "title":titleArticle[random.randint(0,len(titleArticle)-1)]+" "+titleNouns[random.randint(0,len(titleNouns)-1)]+" "+titleAdverbs[random.randint(0,len(titleAdverbs)-1)]+" "+titlePrepositions[random.randint(0,len(titlePrepositions)-1)]+" "+titleNounsPlural[random.randint(0,len(titleNounsPlural)-1)]+" "+titleVerbs[random.randint(0,len(titleVerbs)-1)]+".",
        "tags": thisPostTags,
        "date": str(thisDate)
    }
    posts.insert_one(thisPost)

In [232]:
#now let's look for a record
posts.find_one() #note that if there are multiple documents matching this query, it'll only return the first

{'_id': ObjectId('581b86384182742e9afbe454'), 'date': 'today', 'name': 'Peter'}

In [163]:
#searching for an exact match on a field
posts.find_one({'author':'Jan'})

{'_id': ObjectId('581b5d29418274fe011894bb'),
 'author': 'Jan',
 'date': '2016-11-02',
 'tags': ['fun', 'insight', 'important'],
 'title': 'A industry how inside yams flap.'}

In [235]:
#returning multiple results
for post in posts.find({'author':'Marcia'}): #note that find creates a cursor object, whereas find_one returns a single document
    print (post['title'],"\n")

The election very in horses irritate. 

The comparison only under snakes mourn. 

A discussion just on books tire. 

The year just under horses argue. 

The governor only about candidates remember. 

A election how within homes stretch. 

A effect about about books tire. 

A match where in yams stretch. 

A effect where at yams launch. 

A sea very in ice use. 

A sea about through airplanes unfasten. 

A battle about through fruit relax. 

The kittens just with ice box. 

The nation how around trains borrow. 

The nation only about fruit launch. 



In [150]:
#exclude the _id from the returned results we need to include info in the projections parameter
## note also that this narrows the results further by only including posts by a specific author that contain a specific tag
for post in posts.find({'author':'Marcia','tags':'political'},{'_id':0}):
    print (post,"\n\r")

{'tags': ['political', 'work', 'random'], 'title': 'A year only through homes share.', 'date': '2016-10-28', 'author': 'Marcia'} 

{'tags': ['political', 'fun', 'demo'], 'title': 'The comparison where about books borrow.', 'date': '2016-10-03', 'author': 'Marcia'} 



In [236]:
#update a record
posts.update_one({'author':"Larry"},{'$set':{'author':"Peter"}},upsert=False)

#after updating, look for that record
posts.find_one({'author':'Peter'})

{'_id': ObjectId('581b8726418274fe0118951f'),
 'author': 'Peter',
 'date': datetime.datetime(2016, 11, 3, 18, 51, 18, 637000),
 'tags': ['mongodb', 'demo', 'examples', '760'],
 'title': 'Flippin Flappin Plumbing'}

In [237]:
#to use a regular expression in pymongo as the search parameter takes a bit more work than in the mongo shell.  
# There are a few ways of accomplishing this.  First, let's look at the 'normal' js way

for post in db.posts.find({'tags':{'$regex':'^stup'}}):
    print (post['author'],post['tags'], '\n\r')

Peter ['mongodb', 'stuperheroes', 'examples'] 

Larry ['mongodb', 'stuperheroes', 'examples'] 

Marcia ['important', 'stupid', 'rant'] 

Greg ['fun', 'stupid'] 

Greg ['personal', 'insight', 'stupid'] 

Greg ['random', 'stupid', 'personal'] 

Bobby ['personal', 'stupid'] 

Cindy ['work', 'stupid', 'insight'] 

Cindy ['BYU', 'stupid', 'random'] 

Carol ['fun', 'political', 'stupid'] 

Carol ['stupid', 'fun', 'personal'] 

Alice ['random', 'stupid'] 

Alice ['stupid', 'important', 'political'] 

Alice ['fun', 'rant', 'stupid'] 

Bobby ['random', 'demo', 'stupid'] 

Marcia ['stupid', 'personal', 'random'] 

Mike ['stupid', 'BYU', 'rant'] 

Greg ['personal', 'stupid', 'demo'] 

Jan ['personal', 'stupid', 'political'] 

Greg ['rant', 'stupid', 'work'] 

Marcia ['stupid', 'BYU', 'rant'] 

Bobby ['stupid', 'fun'] 

Alice ['fun', 'stupid', 'work'] 

Alice ['stupid', 'political', 'BYU'] 

Jan ['stupid', 'work', 'rant'] 

Bobby ['760R', 'random', 'stupid'] 

Greg ['political', 'stupid', 'importa

In [168]:
# and now the python way

import re #python's regex library
regx = re.compile("^s",re.IGNORECASE)
for post in db.posts.find({'tags':regx},{'_id':0}):
    print (post, "\n\r")

{'tags': ['mongodb', 'stuperheroes', 'examples'], 'title': 'The amazing Peter Dude', 'date': datetime.datetime(2016, 10, 25, 17, 46, 53, 581000), 'author': 'Peter'} 

{'tags': ['mongodb', 'stuperheroes', 'examples'], 'title': 'The amazing Peter Dude', 'date': datetime.datetime(2016, 10, 25, 17, 48, 0, 770000), 'author': 'Larry'} 

{'tags': ['important', 'stupid', 'rant'], 'title': 'The kittens very at horses share.', 'date': '2016-10-15', 'author': 'Marcia'} 

{'tags': ['fun', 'stupid'], 'title': 'A industry how inside schools bomb.', 'date': '2016-10-04', 'author': 'Greg'} 

{'tags': ['personal', 'insight', 'stupid'], 'title': 'The discussion very through yams box.', 'date': '2016-10-27', 'author': 'Greg'} 

{'tags': ['random', 'stupid', 'personal'], 'title': 'A effect never through airplanes box.', 'date': '2016-10-18', 'author': 'Greg'} 

{'tags': ['personal', 'stupid'], 'title': 'A year very at fruit share.', 'date': '2016-10-13', 'author': 'Bobby'} 

{'tags': ['work', 'stupid', 'i

In [170]:
#let's delete a record using the remove() method
# posts.remove_one({'author':'Peter'})

posts.find_one({'author':'Peter'})

{'_id': ObjectId('580f9a8d41827452127547fc'),
 'author': 'Peter',
 'date': datetime.datetime(2016, 10, 25, 17, 46, 53, 581000),
 'tags': ['mongodb', 'stuperheroes', 'examples'],
 'title': 'The amazing Peter Dude'}

In [184]:
#find all posts written by Bobby OR Cindy
for post in posts.find({'$or':[
            {'author':'Bobby'},{'author':'Cindy'}
        ]},{'_id':0}):
    print (post,'\n\r')

{'tags': ['rant', 'political', 'random'], 'title': 'The discussion very through weather use.', 'date': '2016-10-26', 'author': 'Bobby'} 

{'tags': ['political', 'work', 'fun'], 'title': 'A industry about at weather choke.', 'date': '2016-11-01', 'author': 'Cindy'} 

{'tags': ['insight', 'rant'], 'title': 'A match very under yams unfasten.', 'date': '2016-10-24', 'author': 'Cindy'} 

{'tags': ['work', 'fun', 'demo'], 'title': 'The comparison only about cables tire.', 'date': '2016-10-21', 'author': 'Bobby'} 

{'tags': ['fun', 'BYU'], 'title': 'The election even with trains use.', 'date': '2016-10-05', 'author': 'Cindy'} 

{'tags': ['political', 'insight', 'fun'], 'title': 'The effect only without snakes relax.', 'date': '2016-10-24', 'author': 'Bobby'} 

{'tags': ['fun', 'demo', 'work'], 'title': 'A discovery never inside weather share.', 'date': '2016-10-18', 'author': 'Cindy'} 

{'tags': ['personal', 'stupid'], 'title': 'A year very at fruit share.', 'date': '2016-10-13', 'author': 'B

In [238]:
#change the schema so that all posts in db.posts have an "age" field
posts.update_many({},{'$set':{'age':random.randint(10,60)}})  #this updates all posts with the same value

In [240]:
#let's give each post a different age
cursor = posts.find(modifiers={'$snapshot':True})
index = 0
count = cursor.count()
while index != count:
    doc = cursor[index]
    #update doc
    doc['age']= random.randint(10,60)
    #save changes
    posts.save(doc)
    index +=1
cursor.close()

/Users/peterrich/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead


In [206]:
#Now let's find all the records with an less than 30
for post in posts.find({'age':{'$lt':30}}):
    print (post['author'],post['age'])

Larry 17
Larry 23
Greg 14
Pedro 27
Cindy 20
Cindy 14
Jan 17
Cindy 18
Cindy 25
Pedro 22
Carol 10
Carol 20
Jan 26
Carol 10
Pedro 10
Alice 16
Marcia 17
Alice 10
Mike 13
Bobby 27
Pedro 27
Bobby 25
Greg 16
Alice 11
Mike 14
Cindy 20
Carol 17
Alice 14
Jan 27
Greg 13
Mike 16
Carol 28
Mike 14
Marcia 26
Alice 24
Pedro 28
Bobby 10
Bobby 25
Alice 14
Carol 22
Jan 22
Carol 21


In [241]:
#find all posts with ages between 20 and 30
for post in posts.find({'$and':[{'age':{'$lte':30}},{'age':{'$gte':20}}]}):
    print (post['author'],post['age'])

Marcia 25
Greg 26
Bobby 29
Alice 29
Marcia 25
Alice 23
Bobby 20
Alice 28
Greg 20
Jan 29
Cindy 26
Greg 24
Jan 27
Peter 21
Marcia 25
Alice 24
Cindy 21
Marcia 29
Jan 30
Carol 24
Alice 28
Jan 23
Peter 27
Jan 27


In [222]:
#find posts with only 1 tag in the tags field.  Note that $size uses a specific #
for post in posts.find({'tags':{'$size':2}}):
    print (post['author'],post['tags'])

Mike ['rant', 'BYU']
Cindy ['insight', 'rant']
Alice ['760R', 'insight']
Greg ['fun', 'stupid']
Cindy ['fun', 'BYU']
Carol ['political', '760R']
Jan ['fun', 'demo']
Bobby ['personal', 'stupid']
Jan ['random', 'insight']
Jan ['rant', 'political']
Pedro ['insight', '760R']
Alice ['random', 'stupid']
Alice ['personal', 'random']
Marcia ['rant', 'important']
Bobby ['760R', 'important']
Cindy ['fun', 'demo']
Mike ['demo', 'political']
Alice ['important', 'random']
Bobby ['random', 'BYU']
Mike ['personal', 'insight']
Bobby ['stupid', 'fun']
Cindy ['political', 'personal']
Mike ['personal', 'insight']
Alice ['important', 'fun']


In [220]:
#remove posts with only one tag
posts.delete_many({'tags':{'$size':1}})